# llama_index+
```
pip install llama_index==0.10.20   
pip install llama-index-embeddings-huggingface==0.1.4   
pip install llama-index-llms-huggingface==0.1.4   
pip install llama-index-core==0.10.20.post2  
   
cd BCEmbedding  
pip install -v -e . 
```

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /slurm/home/admin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
from BCEmbedding.tools.llama_index import BCERerank

/slurm/home/admin/.conda/envs/dl/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to
[nltk_data]     /slurm/home/admin/.conda/envs/dl/lib/python3.11/site-
[nltk_data]     packages/llama_index/legacy/_static/nltk_cache...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /slurm/home/admin/.conda/envs/dl/lib/python3.11/site-
[nltk_data]     packages/llama_index/legacy/_static/nltk_cache...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import VectorStoreIndex, ServiceContext, SimpleDirectoryReader
from llama_index.core.node_parser import SimpleNodeParser
from llama_index.core.retrievers import VectorIndexRetriever

In [5]:
# 初始化 embedding模型 和 reranker模型

embed_args = {"model_name": '/slurm/resources/weights/huggingface/maidalun1020/bce-embedding-base_v1', 
              "max_length": 512,
              "embed_batch_size": 32}

embed_model = HuggingFaceEmbedding(**embed_args)

In [6]:
# 初始化 embedding模型 和 reranker模型

reranker_args = {"model": "/slurm/resources/weights/huggingface/maidalun1020/bce-reranker-base_v1",
                  "top_n": 10,}

reranker_model = BCERerank(**reranker_args)

07/27/2024 03:20:24 - [INFO] -BCEmbedding.models.RerankerModel->>>    Loading from `/slurm/resources/weights/huggingface/maidalun1020/bce-reranker-base_v1`.
07/27/2024 03:20:24 - [INFO] -BCEmbedding.models.RerankerModel->>>    Execute device: cuda;	 gpu num: 1;	 use fp16: False


In [7]:
# 提取embeddings

query = "苹果"

passages = [
    "我喜欢苹果",
    "我喜欢橘子",
    "苹果和橘子都是水果"
]

query_embedding = embed_model.get_query_embedding(query)
print("query_embedding shape: ", len(query_embedding))

passages_embeddings = embed_model.get_text_embedding_batch(passages)
print("passages_embeddings shape: ", len(passages_embeddings))


query_embedding shape:  768
passages_embeddings shape:  3


In [9]:
# llm

from llama_index.llms.huggingface import HuggingFaceLLM
import torch

from llama_index.core import PromptTemplate

qa_template = PromptTemplate(
    "以下是描述一个任务的指令。"
    "请根据用户输入的内容，一步一步地回答。\n\n"
    "### 指令:\n{query_str}\n\n### 回复:"
)

llm = HuggingFaceLLM(
    context_window=12000,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.25, "do_sample": False},
    query_wrapper_prompt=qa_template,
    tokenizer_name="/slurm/resources/weights/huggingface/Qwen/Qwen2-7B",
    model_name="/slurm/resources/weights/huggingface/Qwen/Qwen2-7B",
    device_map="auto",
    tokenizer_kwargs={"max_length": 512},
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16}
)

07/27/2024 03:40:02 - [INFO] -accelerate.utils.modeling->>>    We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).
Loading checkpoint shards: 100%|██████████| 4/4 [01:57<00:00, 29.45s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
service_context = ServiceContext.from_defaults(llm=llm, embed_model=embed_model)

/tmp/ipykernel_18834/3999105131.py:1: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(llm=llm, embed_model=embed_model)


In [11]:
documents = SimpleDirectoryReader(input_files=["RAG-编年史.pdf"]).load_data()

In [12]:
len(documents)
documents[0]

Document(id_='4c096d85-144f-473a-bd82-1b590f84d626', embedding=None, metadata={'page_label': '1', 'file_name': 'RAG-编年史.pdf', 'file_path': 'RAG-编年史.pdf', 'file_type': 'application/pdf', 'file_size': 1779723, 'creation_date': '2024-07-25', 'last_modified_date': '2024-07-25'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='Abstract  \n背景：\nRAG是⼀种使⼤型语⾔模型（ LLMs）访问外部数据的流⾏范式。\nRAG的实施⾯临诸多挑战，包括检索模型的有效整合、表示学习、数据多样性、计算效率优化等。\n⽬的：\n提出对巴⻄葡萄⽛语实施、优化和评估 RAG的良好实践。\n通过简单的流⽔线进⾏推理和实验，回答有关哈利 ·波特书籍的问题。\n⽅法：\n使⽤ OpenAI的gpt-4、gpt-4-1106-preview 、gpt-3.5-turbo-1106 和Google的Gemini Pro ⽣成答案。\n重点提升检索器的质量。\n结果：\n⽅法使得 MRR@10 相⽐基线提⾼了 35.4%。\n经过优化，性能进⼀步提升了 2.4%。\n结论：\nRAG架构经过建议的增强后，相对得分从 57.88%提⾼到了 98.61%的最⼤值。\n\xa0\n1 Introduction  \n背景：\nLLMs在AI应⽤中取得显著成绩，尤其在翻译、总结等任务中。\n需要基于最新信息和外部数

In [13]:
node_parser = SimpleNodeParser.from_defaults(chunk_size=1500, chunk_overlap=200)

In [14]:
nodes = node_parser.get_nodes_from_documents(documents)

In [15]:
index = VectorStoreIndex(nodes, service_context=service_context)

In [16]:
query = "万兴科技做什么的?"

In [18]:
vector_retriever = VectorIndexRetriever(index=index, similarity_top_k=3, service_context=service_context)

In [19]:
retrieval_by_embedding = vector_retriever.retrieve(query)

In [20]:
for item in retrieval_by_embedding:
    print(item.text + "\n\n")
    print("-"*100)

4. 由于没有关于 ADA-002 的更多细节，⽂本中提到将这种⽅法仅称为密集检索器。
 
5.1.3 Custom ADA-002  
1. ⾃定义 ADA-002 ：
采⽤了⾃定义 ADA-002 ⽅法，在第 5.1.2节中介绍的密集检索器配置中。
嵌⼊⾃定义在提⾼整体表征中扮演了关键⻆⾊。
2. 嵌⼊⾃定义技术：
不仅限于 OpenAI的嵌⼊，适⽤于其他同类嵌⼊。
有多种⽅法可以优化矩阵，其中之⼀是在未来⼯作中探索的多负例排名损失（ Multiple Negative  
Ranking Loss ）。
3. 微调阶段：
需要两种类型的样本：正样本（标签为 1）和负样本（标签为 -1）。
在数据集中，通常只有正样本，但可以通过简单的随机洗牌⽣成负样本。
最终数据集包含约 400个例⼦，保持 1：3的正负样本⽐例。
4. 影响性能的超参数：
学习率、批量⼤⼩和投影矩阵的维度。
ADA-002 模型有 1536个序列⻓度，投影矩阵的⼤⼩是 1536 x N ，其中 N可以是 1024、2048、4096。
唐国梁Tommy
唐国梁Tommy
唐国梁Tommy
唐国梁Tommy
唐国梁Tommy
唐国梁Tommy
唐国梁Tommy
唐国梁Tommy
唐国梁Tommy
唐国梁Tommy
唐国梁Tommy
唐国梁Tommy
唐国梁Tommy
唐国梁Tommy
唐国梁Tommy


----------------------------------------------------------------------------------------------------
2. 实施影响：
选择的 BM25实现会影响效果。
Pyserini的BM25实现包括预处理步骤，如词⼲提取和特定语⾔的停⽤词去除。
为了⽐较，包括了使⽤ rank-bm25 的结果，这是⼀种没有预处理的基础实现，并且在 Python中⼴泛使
⽤，集成到像 LangChain 和Llama-index 这样的库中。
3. Pyserini BM25 的应⽤：
在所有实验中都使⽤了 Pyserini BM25 实现，考虑   和  。
 
5.1.2 ADA-002  
1. ADA-002 架构：
OpenAI没有公开 ADA-002 架构的详细信息。
尽管如此，该模型在检索过程中被⽤作所展示的

In [21]:
retrieval_by_reranker = reranker_model.postprocess_nodes(retrieval_by_embedding, query_str=query)

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [22]:
for item in retrieval_by_reranker:
    print(item.text + "\n\n")
    print("-"*100)

2. 实施影响：
选择的 BM25实现会影响效果。
Pyserini的BM25实现包括预处理步骤，如词⼲提取和特定语⾔的停⽤词去除。
为了⽐较，包括了使⽤ rank-bm25 的结果，这是⼀种没有预处理的基础实现，并且在 Python中⼴泛使
⽤，集成到像 LangChain 和Llama-index 这样的库中。
3. Pyserini BM25 的应⽤：
在所有实验中都使⽤了 Pyserini BM25 实现，考虑   和  。
 
5.1.2 ADA-002  
1. ADA-002 架构：
OpenAI没有公开 ADA-002 架构的详细信息。
尽管如此，该模型在检索过程中被⽤作所展示的双编码器设计。
2. 双编码器设计：
为所有可⽤的块构建向量表示。
每个输⼊查询在搜索时计算其嵌⼊。
随后，使⽤余弦相似性评估查询和块之间的相似度。
3. 图5说明：
展示了双编码器架构，其中包括输⼊ 1的查询词标和输⼊ 2的块词标。
两个输⼊在多层架构中各⾃编码，然后通过余弦相似度来评估它们之间的相似性。
唐国梁Tommy
唐国梁Tommy
唐国梁Tommy
唐国梁Tommy
唐国梁Tommy
唐国梁Tommy
唐国梁Tommy
唐国梁Tommy
唐国梁Tommy
唐国梁Tommy
唐国梁Tommy
唐国梁Tommy
唐国梁Tommy
唐国梁Tommy
唐国梁Tommy


----------------------------------------------------------------------------------------------------
展示了混合 BM25-ADA-002 ⽅法的检索器⽐较。
其中包括平均倒数排名（ MRR）和召回率（ R@k）。
实验中，只有 Pyserini的BM25被测试为稀疏检索器，⽽ ADA-002 和⾃定义 ADA-002 被测试为密集检索
器。
提供了最佳结果的混合组合是使⽤ BM25和⾃定义 ADA-002 。
 
5.1.5 Reranker 重排  
1. 重排器的基本概念：
多阶段排名的基础是将⽂档排名分为⼀系列阶段，每个后续阶段重新评估并排名前⼀阶段传递来的候选
集。
2. 多阶段排名：
图9展示了⼀个多阶段管道，其中 Pyserini BM25 负责第⼀阶段，然后候选块由重排器重新评估。
重排后

### 查询引擎：方式-1

In [23]:
query_engine = index.as_query_engine(node_postprocessors=[reranker_model])
query_response = query_engine.query(query)

/slurm/home/admin/.conda/envs/dl/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.25` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


In [24]:
print(str(query_response))

 

根据提供的上下文信息，万兴科技是一家专注于开发和销售数字创意软件的公司。该公司提供一系列软件产品，包括视频编辑、音频编辑、图像编辑、文档编辑、PDF编辑、动画制作、图形设计、音乐制作和数字营销等。这些软件产品旨在帮助用户轻松地创建、编辑和分享数字内容。


### 查询引擎：方式-2

In [25]:
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine.from_args(vector_retriever, llm=llm)

In [26]:
response = query_engine.query(query)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


In [27]:
print(str(response))

 

根据提供的上下文信息，万兴科技是一家专注于开发和销售数字创意软件的公司。该公司提供一系列软件产品，包括视频编辑、音频编辑、图像编辑、文档编辑、PDF编辑、动画制作、图形设计、音乐制作和数字营销等。这些软件产品旨在帮助用户轻松地创建、编辑和分享数字内容。
